In [1]:
from rdflib.plugins.stores import sparqlstore
import rdflib
from rdflib import URIRef, Literal, Graph, Dataset

import serialization
import discourse

import html
import loader
import discourse
import pandas as pd
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
from IPython.core.display import HTML

import uuid

from itertools import chain

def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()

    display(Image(png))

def t2rdflibg(triples): #triples to rdflib graph
    g = Graph()
    for t in triples:
        g.add(t)
    return g


def nan2None(value):
    if pd.isnull(value) :
        return None
    else:
        return value


def get_data_rows(filename):
    data_rows_df = pd.read_csv(filename)
    data_rows=[]
    for i,r in data_rows_df.iterrows():
        r_d = {k:nan2None(v) for k,v in dict(r).items()}

        data_rows.append(r_d)
    del data_rows_df
    return data_rows

serial_key = { "Business Domain" : "ModelDomain", 
               "Model" : "Model", 
               "ModelType" : "ModelType", 
               "Entity" : "Class", 
               "Attribute" : "Attribute", 
               "DataType" : "DataType", 
               "Context" : "Context", 
               "SubContext" : "SubContext", 
               "Relationship" : "Relationship", 
               "Entity From" : "FromClass",
               "Attribute From" : "FromAttribute", 
               "Cardinality From" : "FromCardinality",
               "Entity To" : "ToClass",
               "Attribute To" : "ToAttribute",
               "Cardinality To" : "ToCardinality"
}

jena = sparqlstore.SPARQLUpdateStore("http://localhost:3030/modelg/query",context_aware=True)
#jena.open(("http://localhost:3030/modelg/query", "http://localhost:3030/modelg/update"))
jena.open(("http://localhost:3030/models/query", "http://localhost:3030/models/update"))

# Create a Dataset
ds = Dataset(store=jena, default_union=True, default_graph_base="http://base.raw")
#ds = Dataset(store=jena, default_union=True)
# Define a named graph within the dataset, this will contain all nodes contained within the graph referenced, or none if it doesn't exist yet.
sg = ds.graph(URIRef("http://config"))
sg = Graph()
master_g = ds.graph(URIRef("http://master"))
discourse_g = ds.graph(URIRef("http://discourse"))



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MappingMetaTarget belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, owl.topObjectProperty]; I'm trying to fix it...
* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit:/usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/rd/q3bhk6_n56x75m05pp2sq7dr0000gn/T/tmpj9x_2vs5


Serialization.Meta [Serialization.SerializationMetaData]
Serialization.Mapping [Serialization.SerializationSpecs]
Serialization.BatchDefinition [Serialization.SerializationSpecs]
Serialization.SerializationSpecs [Serialization.SerializationMetaData]
Serialization.BatchNode [Serialization.DataLoad]
Serialization.DataLoad [Serialization.SerializationMetaData]
Serialization.SerializationMetaData [owl.Thing]
Serialization.DataPropertyMapping [Serialization.Mapping]
Serialization.MetaDataProperty [Serialization.Meta]
Serialization.EntityMapping [Serialization.Mapping]
Serialization.MetaClass [Serialization.Meta]
Serialization.JobNode [Serialization.DataLoad]
Serialization.MetaProperty [Serialization.Meta]
Serialization.PropertyMapping [Serialization.Mapping]
Serialization.RowNode [Serialization.DataLoad]
Serialization.Serialization [Serialization.SerializationSpecs]


* Owlready2 * HermiT took 0.7655632495880127 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [2]:
model_data = pd.read_csv(r"mdm_v11_3_pdm_additional.csv").fillna("")
model_data.columns=['Sequence', 'ModelDomain', 'Model', 'ModelType', 'Class',
       'ClassDescription', 'Attribute', 'AttributeDescription', 'DataType',
       'Context', 'SubContext', 'Nulls', 'Is PK']

In [3]:
def key_filter(keys,xlist=None):
    s_keys = sorted(keys)
    if xlist is not None:
        if not tuple([k for k in s_keys if k not in xlist]) == tuple():
            s_keys = tuple([k for k in s_keys if k not in xlist])
        else:
            s_keys = tuple(s_keys)

    else:
        s_keys = tuple([k for k in s_keys])
    return s_keys

from functools import partial
from collections import Counter
fkf = partial(key_filter, xlist=["LANG_TP_CD"])

In [4]:
raw_keys = dict(model_data[model_data['Is PK']=="Yes"].groupby("Class")['Attribute'].agg(fkf))
[(k,v) for k,v in Counter(raw_keys.values()).items() if len(k)>1]

[(('BKTHASH', 'RECNO', 'SRCRECNO'), 1),
 (('CMPSEQNO', 'RECNO'), 1),
 (('SRCID', 'SRCRECNO'), 1),
 (('CITY_NAME',
   'GIVEN_NAME_ONE',
   'LAST_NAME',
   'P_CITY_NAME',
   'P_GIVEN_NAME_ONE',
   'P_LAST_NAME'),
  1)]

In [5]:
# Contents here block the inversion of the table/pk mapping dict
print([(k,v) for k,v in Counter(raw_keys.values()).items() if v>1])
# FOR IBM products, sharing the same PK implies parent/sub-class relationships (though there's)
# no systematic method for determining which one is the parent.
sorted([(k,v) for k,v in raw_keys.items() if v in [k for k,v in Counter(raw_keys.values()).items() if v>1]], key=lambda x : x[1])
manual_key_masters = { ('CONT_ID',) : 'CONTACT', 
                       ('HOLDING_ID',): 'HOLDING', 
                      ('LOCATION_GROUP_ID',) : 'LOCATIONGROUP', 
                      ('PAYMENT_SOURCE_ID', ): 'PAYMENTSOURCE',
                      ('PPREF_ID',) : 'PRIVPREF',
                      ('PRODUCT_ID',) : 'PRODUCT', 
                      ('SUSPECT_ID',) : 'SUSPECT' , 
                      ('LANG_TP_CD', ) : 'CDLANGTP'
                     }

key_map = {**{v:k for k,v in raw_keys.items()}, **manual_key_masters}


[(('LOCATION_GROUP_ID',), 3), (('PAYMENT_SOURCE_ID',), 4), (('CONT_ID',), 6), (('PRODUCT_ID',), 5), (('HOLDING_ID',), 3), (('PPREF_ID',), 3), (('SUSPECT_ID',), 2)]


In [6]:
def att_link(row):
    attribute=row['Attribute']
    table = row['Class']
    t_att = tuple([attribute])
    t_val = key_map.get(t_att)
    relationship = str(table) + " --> " + str(t_val)
    if t_val is not None and t_val != table:
        return pd.Series((relationship, table, attribute, t_val, t_att[0]))
    else:
        return pd.Series((None, None, None, None, None))
    

In [7]:
# Extend model_data to include extracted relationship information
model_data[['Relationship','FromClass','FromAttribute','ToClass','ToAttribute']] = model_data.apply(att_link, axis=1)
#model_data

In [8]:
def meta_data_package_template(field_d):
    dc_terms_base = "http://purl.org/dc/terms/"
    rdf_form = {}
    for k,v in field_d.items():
        rdf_form[URIRef(dc_terms_base + k)] = Literal(v)
    return rdf_form



d_p = meta_data_package_template( {"created" : "2023-04-01", 
                                   "creator" : "tomk", 
                                   "description" : "test data package", 
                                   "title" : "some title", 
                                   "modified" : "2023-04-05"})



In [9]:
rows = [dict({rk:html.escape(str(rv)) for rk, rv in r.items()}) for i,r in model_data.iterrows()]

In [10]:
d_p

{rdflib.term.URIRef('http://purl.org/dc/terms/created'): rdflib.term.Literal('2023-04-01'),
 rdflib.term.URIRef('http://purl.org/dc/terms/creator'): rdflib.term.Literal('tomk'),
 rdflib.term.URIRef('http://purl.org/dc/terms/description'): rdflib.term.Literal('test data package'),
 rdflib.term.URIRef('http://purl.org/dc/terms/title'): rdflib.term.Literal('some title'),
 rdflib.term.URIRef('http://purl.org/dc/terms/modified'): rdflib.term.Literal('2023-04-05')}

In [11]:
if len(sg)==0:
    sg.parse("DMEAR_ser.rdf")

In [12]:
len(list(sg.triples((None, None, None))))

207

In [13]:
S = serialization.Serialization(sg, "EntityAttributeRecord")

In [14]:
# Here master_g is a master graph sourced from Fuseki
mq,me_m = loader.get_triples(sg, "EntityAttributeRecord", rows, master_g, [])

0:00:09.475771 for 1944


In [15]:
p,de,di = loader.generate_discourse("test_123", mq, d_p)

In [16]:
def triples_to_quads(triples, graph="http://master"):
    for s,p,o, *_ in triples:
        yield (s,p,o,URIRef(graph))

In [17]:
q_gen = triples_to_quads(mq, "http://master")

In [18]:
q_quads = list(q_gen)

In [19]:
len(mq), len(p), len(de), len(di)

(13775, 96425, 55100, 13783)

In [20]:

ds.addN(triples_to_quads(mq, URIRef("http://master")))
ds.addN(triples_to_quads(p, URIRef("http://discourse")))
ds.addN(triples_to_quads(de, URIRef("http://discourse")))
ds.addN(triples_to_quads(di, URIRef("http://discourse")))


In [21]:
ds.addN(triples_to_quads( [(URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/236472ed5205428f91f080ac2962229b'), 
  URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#DiscourseContains'),
  URIRef("disco_test")),
  (URIRef("disco_test"),
  URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#DiscourseContains'),
   URIRef("disco_test_thing")),
 (URIRef("disco_test_thing"),URIRef("rdf:type"),URIRef("itchyness"), 
 )
], URIRef("http://discourse") ))

In [22]:


qr = discourse_g.query("""select ?s ?p ?o {

 ?s ?p ?o. 
 VALUES ?p {rdf:type}

}""")



In [23]:
import owlready2 as owlr
disco = owlr.get_ontology("discourse.owl").load()
disco_ns = Namespace(disco.base_iri)
discourse_g.bind('disco', disco_ns, override=True, replace=True)

In [24]:

qr = discourse_g.query("""select ?s ?p ?o {

 ?discourse rdf:type disco:Discourse. 
 ?s ?p ?o.
 FILTER ( ?p = disco:DiscourseContains)

}""")



In [32]:

qr = discourse_g.query("""select ?discourse ?o ?t {

 ?discourse rdf:type disco:Discourse. 
 ?discourse disco:DiscourseContains+ ?o .
 ?o rdf:type ?t.

}""")



In [26]:
def pull_last_values(v):
    if isinstance(v, rdflib.URIRef):
        return v.n3().split("/")[-1][:-1]
    else:
        return type(v)

In [27]:
pd.DataFrame([tuple(pull_last_values(p) for p in q) for q in qr])

,0,1,2
0,5e006e98e3b6430a8b5b260426266922,48deac3462e74284b9cfcac12050aa0c,discourse#Declaration
1,5e006e98e3b6430a8b5b260426266922,5c49be2c796a459f8f7e3999c6b7b30e,discourse#Declaration
2,5e006e98e3b6430a8b5b260426266922,c31ab1a04c464304aadf5a373076a653,discourse#Declaration
3,5e006e98e3b6430a8b5b260426266922,1536d26b1b7e42f7a5bc04484bdc2397,discourse#Declaration
4,5e006e98e3b6430a8b5b260426266922,62f51c24b3e4455fa36aa7466e884608,discourse#Declaration
...,...,...,...
68870,236472ed5205428f91f080ac2962229b,5fd8a3d606ed40ba864c479a6bdf80f0,discourse#Declaration
68871,236472ed5205428f91f080ac2962229b,ca156c132a72490597e96a962e7ce7b4,discourse#Declaration
68872,236472ed5205428f91f080ac2962229b,fbf542e01e49423485ee6ed332170084,discourse#Declaration
68873,236472ed5205428f91f080ac2962229b,39579724cb2d447a8be28db79bca42e0,discourse#Declaration


In [28]:
Counter([o for s,p,o in qr])

Counter({rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#Declaration'): 68875})

In [29]:
tg=ds.triples((None, None, None))

In [30]:
next(tg)

(rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'))

In [31]:
disco_ns

Namespace('http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#')